In [ ]:
import os, sys
from numpy import asarray, zeros, arange, random
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [2]:
TEXT_DATA_DIR = 'D://20_newsgroup/'
GLOVE_DIR = 'D://glove.6B/glove.6B.100d.txt'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32


In [3]:
#load tests
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                texts.append(f.read())
                f.close()
                labels.append(label_id)
print('Found %s texts.' % len(texts))


Found 19997 texts.


In [4]:
#load words embedding vector dict
embeddings_index = dict()
f = open(GLOVE_DIR, 'r', encoding='UTF-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 400000 word vectors.


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 214873 unique tokens.


In [6]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [7]:
# split the data into training and validation set
indices = arange(data.shape[0])
random.shuffle(indices)
data = data[indices]
labels = asarray(labels)[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)


(15998, 1000)
(15998, 20)
(3999, 1000)
(3999, 20)


In [8]:
#embedding matrix
#nb_words = max(MAX_NB_WORDS, len(word_index))
nb_words = len(word_index)
embedding_matrix = zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    #if i > MAX_NB_WORDS:
    #    continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print(embedding_matrix.shape)


(214874, 100)


In [9]:
#build model
model = Sequential()
model.add(Embedding(nb_words + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(labels_index), activation='softmax'))
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())
print(data.shape)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         21487400  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2020      
Total params: 21,569,820
Trainable params: 82,420
Non-trainable params: 21,487,400
_________________________________________________________________
None
(19997, 1000)


In [10]:
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=15,
          validation_data=(x_val, y_val))


Train on 15998 samples, validate on 3999 samples
Epoch 1/15
15998/15998 [==============================] - 370s 23ms/step - loss: 2.6036 - acc: 0.1692 - val_loss: 2.0816 - val_acc: 0.3021
Epoch 2/15
15998/15998 [==============================] - 368s 23ms/step - loss: 1.9007 - acc: 0.3495 - val_loss: 1.4688 - val_acc: 0.4819
Epoch 3/15
15998/15998 [==============================] - 374s 23ms/step - loss: 1.4007 - acc: 0.5177 - val_loss: 1.0773 - val_acc: 0.6389
Epoch 4/15
15998/15998 [==============================] - 399s 25ms/step - loss: 0.8532 - acc: 0.7234 - val_loss: 0.5017 - val_acc: 0.8407
Epoch 5/15
15998/15998 [==============================] - 386s 24ms/step - loss: 0.4241 - acc: 0.8801 - val_loss: 0.2152 - val_acc: 0.9400
Epoch 6/15
15998/15998 [==============================] - 374s 23ms/step - loss: 0.2500 - acc: 0.9325 - val_loss: 0.1849 - val_acc: 0.9447
Epoch 7/15
15998/15998 [==============================] - 385s 24ms/step - loss: 0.1885 - acc: 0.9449 - val_loss: 0.1